## 1.5 An Extended Example: Tic-Tac-Toe

### Setup

<img src="tic_tac_toe.png" align="left" height="10" width="150" style="margin-right: 100px">

A imperfect player, who only makes random move, plays against a mostly greedy player, selecting a the move that leads to the state with greatest value.  
Occasionally, however, the latter player selects moves that randomly. These are called _exploratory_ moves because they cause us to  experience states that he might otherwise never see.



<br>
<br>

<img src="back_up.png" height="10" width="500" align="right" style="margin-right: 200px">

### Algorithm

Here is an example of how tic-tac-toe would be approached with a method making use of a value function:
1. We initialise a table of numbers, one for each state of the board. The state of the board is represented as a string by reading all positions row by row and concatenating them.
   The value of the state is the probability of winning if the board is in that specific state.
   E.g. {"XXX O 0 0": 0.5} 
2. We start playing against our opponent. Every time we need to make a move, we:  
   2.1 List all the possible move  
   2.2 Lookup the __state-value__ of each move from the __state-value__ dictionary  
   2.3 Select the move with the highest __state-value__, i.e. the highest probability of winning  
   2.4 We inform the previous state about the new probability of winning, updating its value using _temporal difference_ learning:
   \begin{equation}
       V(S_t) \leftarrow V(S_t) + \alpha[V(S_{t+1}) - V(S_t)]
   \end{equation}
   
   
The diagram on the left show as an example of backup update via temporal difference.  
Note that the exploratory move (the one not marked with the `*`) does not result into value update.

## Let's setup the environment

The board is represented as a flat list of 9 positions, reading row by row, from left to right.
An empty cell is represented by a whitespace `' '`.
A scheme below:

```
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
```

Here is an example of a game at a random state, in which the two players use, `X`s and `O`s:
```
[' ', ' ', 'O']
[' ', ' ', 'O']
['X', 'X', 'X']
```


In [ ]:
class TicTacToe:
    def __init__(self, player, opponent):
        """
        Initialises a Tic-Tac-Toe environment, by specifying the two players in the game.
        """
        self.board = [" " for x in range(9)]
        self.player = player
        self.opponent = opponent
    
        # init
        self.player_turn = True
        self.moves_left = 9
        return
    
    def reset(self):
        """
        Resets the environment to the initial state: an empty board
        """
        self.board = [" " for x in range(9)]
        self.moves_left = 9
        return self.board, False
    
    def step(self):
        """
        Performs one play for the player that holds its turn.
        This updates the board with the new move.
        """
        if self.player_turn:
            player = self.player
        else:
            player = self.opponent
        
        pos, reward = player.get_action(self.board)
        self.board[pos] = player.marker
        
        self.player_turn = not self.player_turn
        self.moves_left -= 1

        return self.board, reward, self.game_ended()
    
    def learn(self, n_games):
        """
        Plays `n_games` consecutively to let the player learn.
        Args:
            n_games (int): the number of games to learn from
        """
        self.player.train()
        for i in range(n_games):
            print("Playing game {}\t".format(i), end="\r")
            state, done = self.reset()
            while not done:
                state, reward, done = self.step()
        return self.player
    
    def play(self):
        """
        Tests the skills of each player in the game by having a single game, whilst learning is disabled.
        """
        self.player.eval()
        state, done = self.reset()
        while not done:
            state, reward, done = self.step()
            self.draw()
        return state, done
    
    def game_ended(self):
        """
        Checks if the game reached its conclusing, either because there are no moves left,
        or because one of the two player has won.
        """
        if self.moves_left <= 0:
            return True
        
        state = self.board
        # row
        for i in range(0, 9, 3):
            if (state[i] == state[i + 1] == state[i + 2]):
                return state[i] != " "
        # col
        for i in range(3):
            if (state[i] == state[i + 3] == state[i + 6]):
                return state[i] != " "
        # diag
        if (state[0] == state[4] == state[8]):
            return state[0] != " "
        # anti-diag
        if (state[2] == state[4] == state[6]):
            return state[2] != " "
        
        return False        
    
    def draw(self):
        """
        Plots the current state of the board
        """
        for i in range(0, 9, 3):
            print(self.board[i:i + 3])
        print("State value:", self.player.get_value(self.board))
        print()

### Now, let's setup an imperfect player, who can only make random moves

In [2]:
import random
import copy
import numpy


class RandomPlayer:
    def __init__(self, marker):
        self.marker = str(marker)
    
    @staticmethod
    def get_action(state):
        """
        Given the current state of the board, selects a random move from all the available moves
        """
        pos = random.randint(0, 8)
        while state[pos] != " ":
            pos = random.randint(0, 8)
        return pos, None

### And an E-Greedy player

The e-greedy player makes greedy moves, i.e. moves whose value is max among the list of all possibile moves.  
Occasionally, with probability `e`, it makes an _exploratory_ move, a move that would not experience otherwise.

In [28]:
class EGreedyPlayer:
    def __init__(self, marker, init_value=0.5, e_greedy=0., step_size=0.5,
                 decrement=0.9,
                 decrement_each=1000,
                 learn=True):
        self.marker = str(marker)
        self.learn = learn
        self.e_greedy = e_greedy
        self.step_size = step_size
        self.init_value = init_value
        self.decrement = decrement
        self.decrement_each = decrement_each
        
        # we store each state of the board as a dict whose key is the hash of the list
        # and the value is its state value
        self.state_value = {}
        self.n_plays = 0
        return
    
    def train(self):
        """
        Enables updates of the state-value table while playing
        """
        self.learn = True
        
    def eval(self):
        """
        Disables updates of the state-value table while playing
        """
        self.learn = False
    
    def encode(self, state):
        """
        Encodes the state of the board into a single string
        Args:
            state (List[str]): The board state as a list of strings: the player markers
        """
        return "".join(state)
    
    def set_value(self, state, value):
        """
        Updates the state-value of the given state in the table
        Args:
            state (List[str]): The board state as a list of strings: the player markers
            value (float): the probability of winning in that state
        """
        encoded = self.encode(state)
        self.state_value[encoded] = value
        return 
    
    def get_value(self, state):
        """
        Performs a lookup to the state-value table for the given state.
        Args:
            state (List[str]): The board state as a list of strings: the player markers
        Returns:
            (float): the value of the state, if already visited, the initial value otherwise
        
        """
        encoded = self.encode(state)
        if encoded in self.state_value:
            return self.state_value[encoded]
        else:
            return self.init_value
    
    def back_up(self, state, next_state, next_value):
        """
        Performs a backup update of the current state using temporal difference learning
        Args:
            state (List[str]): The current state of the board as a list of strings: the player markers
            next_state (List[str]): The state of the board if we had to make this move, as a list of strings
            next_value (float): the value of the next state, if we had to make this move
        """
        current_value = self.get_value(state)           
        self.set_value(next_state, next_value)

        # TD update
        new_value = current_value + self.step_size * (next_value - current_value)
        self.set_value(state, new_value)
        return
    
    def get_action(self, state):
        """
        Computes the e-greedy action, according to the current state of the board.
        Args:
            state (List[str]): The current state of the board
        Returns:
            (Tuple[int, float]): A tuple containing the best move and the reward deriving from it
        """
        # perform exploratory move with probability self.e_greedy
        # do not update value table
        if random.random() < self.e_greedy and self.learn:
            return RandomPlayer.get_action(state)
        
        best_move = None
        best_state = None
        best_reward = -float("inf")
        for pos in range(len(state)):
            # check that the cell is free
            if state[pos] == " ":    
                next_state = copy.deepcopy(state)
                next_state[pos] = self.marker
                reward = self.get_reward(next_state)

                # choose greedily
                if reward >= best_reward:
                    best_move = pos
                    best_state = next_state
                    best_reward = reward
        
        if self.learn:
            self.back_up(state, best_state, best_reward)
        
        self.n_plays += 1
        if self.n_plays % self.decrement_each == 0:
            self.step_size *= self.decrement
        
        return best_move, best_reward
    
    def get_reward(self, state):
        """
        Computes the reward of the player at a given state of the board
        Args:
            state (List[str]): The current state of the board
        Returns:
            (float): 1 if the move is a winning move, 0 is the state is making the player lose.
                     In all the other cases returns the probability of the current state as stored in the 
                     state-value table.
        """
        # row
        for i in range(0, 9, 3):
            if (state[i] == state[i + 1] == state[i + 2]):
                # three in a row
                if state[i] == self.marker:
                    return 1  # won, win prop 1.
                elif state[i] == " ":
                    return self.get_value(state)  # row is empty
                else:
                    return 0  # lost, win prob 0.       
        # col
        for i in range(3):
            if (state[i] == state[i + 3] == state[i + 6]):
                # three in a row
                if state[i] == self.marker:
                    return 1  # won, win prop 1.
                elif state[i] == " ":
                    return self.get_value(state)  # row is empty
                else:
                    return 0  # lost, win prob 0.                
        # diag
        if (state[0] == state[4] == state[8]):
            if not self.learn:
                print("diag")
            # three in a row
            if state[0] == self.marker:
                return 1  # won, win prop 1.
            elif state[0] == " ":
                return self.get_value(state)  # row is empty
            else:
                return 0  # lost, win prob 0.
        # anti-diag
        if (state[2] == state[4] == state[6]):
            # three in a row
            if state[2] == self.marker:
                return 1  # won, win prop 1.
            elif state[2] == " ":
                return self.get_value(state)  # row is empty
            else:
                return 0  # lost, win prob 0.
        return self.get_value(state)

In [29]:
opponent = RandomPlayer("O")
player = EGreedyPlayer("X", init_value=0.5, e_greedy=0.3, step_size=0.5, decrement=0.8, decrement_each=10000)
env = TicTacToe(opponent=opponent, player=player)

In [30]:
env.learn(100000)

In [46]:
env.play()

[' ', 'O', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']
State value: 0.5

[' ', 'O', ' ']
[' ', ' ', ' ']
[' ', ' ', 'X']
State value: 0.5

[' ', 'O', 'O']
[' ', ' ', ' ']
[' ', ' ', 'X']
State value: 0.999166204770979

['X', 'O', 'O']
[' ', ' ', ' ']
[' ', ' ', 'X']
State value: 0.999197888318448

['X', 'O', 'O']
[' ', ' ', ' ']
[' ', 'O', 'X']
State value: 0.9999999827065554

diag
['X', 'O', 'O']
[' ', 'X', ' ']
[' ', 'O', 'X']
State value: 1



(['X', 'O', 'O', ' ', 'X', ' ', ' ', 'O', 'X'], True)

In [14]:
player.step_size

0.002951479051793532